<a href="https://colab.research.google.com/github/dswh/lil_nlp_with_tensorflow/blob/main/02_02_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings for Sentiment Analysis

This notebook explains an introduction to word embeddings. We will train our own word embeddings using a simple Keras model for a sentiment classification task.

Steps include:
1. Downloading data from tensorflow dataset.
2. Segregating training and testing sentences & labels.
3. Data preparation to padded sequences
4. Defining out Keras model with an Embedding layer.
5. Train the model and explore the weights from the embedding layer.


In [ ]:
##import the required libraries and APIs
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.4.1


## Downloading the TensorFlow `imdb_review` dataset

> Make sure tensorflow_datasets is installed

In [ ]:
##load the imdb reviews dataset
data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

## Segregating training and testing sets

In [ ]:
##segregate training and test set
train_data, test_data = data['train'], data['test']

##create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [ ]:
##iterate over the train data to extract sentences and labels
for sent, label in train_data:
    train_sentences.append(str(sent.numpy().decode('utf8')))
    train_labels.append(label.numpy())

##iterate over the test set to extract sentences and labels
for sent, label in test_data:
    test_sentences.append(str(sent.numpy().decode('utf8')))
    test_labels.append(label.numpy())



In [ ]:
##convert lists into numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Data preparation - setting up the tokenizer

In [ ]:
##define the parameters for the tokenizing and padding
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

##training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs,maxlen=max_length, truncating=trunc_type)

##testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[   0    0    0    0    0    0    0    0   11   26   75  571    6  805
 2354  313  106   19   12    7  629  686    6    4 2219    5  181  584
   64 1454  110 2263    3 3951   21    2    1    3  258   41 4677    4
  174  188   21   12 4078   11 1578 2354   86    2   20   14 1907    2
  112  940   14 1811 1340  548    3  355  181  466    6  591   19   17
   55 1817    5   49   14 4

## Define the Neural Network with Embedding layer

1. Use the Sequential API.
2. Add an embedding input layer of input size equal to vocabulary size.
3. Add a flatten layer, and two dense layers.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

##compile the model with loss function, optimizer and metrics
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [ ]:
num_epochs = 10

##train the model with training and validation set
model.fit(
    train_padded, 
    train_labels, 
    epochs=num_epochs, 
    validation_data=(test_padded, test_labels)
    )

Epoch 1/10
782/782 [==============================] - 5s 5ms/step - loss: 0.6209 - accuracy: 0.6148 - val_loss: 0.3611 - val_accuracy: 0.8405
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2431 - accuracy: 0.9077 - val_loss: 0.3744 - val_accuracy: 0.8386
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0871 - accuracy: 0.9803 - val_loss: 0.4643 - val_accuracy: 0.8239
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0266 - accuracy: 0.9970 - val_loss: 0.5328 - val_accuracy: 0.8282
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0109 - accuracy: 0.9988 - val_loss: 0.5722 - val_accuracy: 0.8275
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0066 - accuracy: 0.9991 - val_loss: 0.6322 - val_accuracy: 0.8233
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0036 - accuracy: 0.9995 - val_loss: 0.6825 - val_accuracy: 0.8258
Epoch 

## Deriving weights from the embedding layer

In [ ]:
##isolating the first embedding layer
l1 = model.layers[0]

##extracting learned weights
weights = l1.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
print(weights[0])

(10000, 16)
[-0.10705623  0.01728459  0.01023278 -0.01320995  0.01645457  0.02442149
  0.03636202  0.02251814  0.00503969  0.02718268 -0.01034018 -0.01450449
  0.01457764 -0.0747593  -0.01561933 -0.08514806]
